In [1]:
import requests
import pandas as pd
import datetime
import configparser
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np

In [2]:
config_data=configparser.ConfigParser()
config_data.read("config.ini")
keys=config_data['keys']
client=Client(keys['apiKey'],keys['secretKey']) #create binance api client
print('logged in')

logged in


In [3]:
# Calculate the start and end dates for yesterday
end_date = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
start_date = end_date - datetime.timedelta(days=1)

# Convert the dates to milliseconds for the Binance API
start_timestamp = int(start_date.timestamp() * 1000)
end_timestamp = int(end_date.timestamp() * 1000)

In [4]:
historical = client.get_historical_klines('BTCUSDT',Client.KLINE_INTERVAL_1DAY,start_timestamp, end_timestamp)# parameters:(symbol, interval, date_from)
historical

[[1685318400000,
  '28065.01000000',
  '28447.14000000',
  '27524.60000000',
  '27736.40000000',
  '42385.41945000',
  1685404799999,
  '1182678581.63466910',
  805579,
  '20199.85331000',
  '563722083.65919410',
  '0']]

In [5]:
hist_df=pd.DataFrame(historical)
hist_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1685318400000,28065.01000000,28447.14000000,27524.60000000,27736.40000000,42385.41945000,1685404799999,1182678581.63466910,805579,20199.85331000,563722083.65919410,0


In [6]:
hist_df.columns=['date','open','high','low','close','volume','Close Time','Quote Asset Volume','Number of Trades','TB Base Volume','TB Quote Volume','Ignore']
hist_df.head()

,date,open,high,low,close,volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,1685318400000,28065.01000000,28447.14000000,27524.60000000,27736.40000000,42385.41945000,1685404799999,1182678581.63466910,805579,20199.85331000,563722083.65919410,0


In [7]:
#convert from Unix timestamps to date
hist_df['date']= pd.to_datetime(hist_df['date']/1000, unit='s')
hist_df['Close Time']= pd.to_datetime(hist_df['Close Time']/1000, unit='s')
hist_df.dtypes

date                  datetime64[ns]
open                          object
high                          object
low                           object
close                         object
volume                        object
Close Time            datetime64[ns]
Quote Asset Volume            object
Number of Trades               int64
TB Base Volume                object
TB Quote Volume               object
Ignore                        object
dtype: object

In [8]:
hist_df.head() #notice open time and close time 

,date,open,high,low,close,volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2023-05-29,28065.01000000,28447.14000000,27524.60000000,27736.40000000,42385.41945000,2023-05-29 23:59:59.999000064,1182678581.63466910,805579,20199.85331000,563722083.65919410,0


In [9]:
hist_df.date=hist_df.date.apply(lambda x: x.date())

In [10]:
numeric_columns=['open','high','low','close','volume','Quote Asset Volume','TB Base Volume','TB Quote Volume']
hist_df[numeric_columns]=hist_df[numeric_columns].apply(pd.to_numeric,axis=1)
hist_df.head()

,date,open,high,low,close,volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2023-05-29,28065.01,28447.14,27524.6,27736.4,42385.41945,2023-05-29 23:59:59.999000064,1.182679e+09,805579,20199.85331,5.637221e+08,0


In [11]:
hist_df.drop(['Close Time','Quote Asset Volume','TB Quote Volume','Ignore'],inplace=True,axis=1)

In [12]:
hist_df

,date,open,high,low,close,volume,Number of Trades,TB Base Volume
0,2023-05-29,28065.01,28447.14,27524.6,27736.4,42385.41945,805579,20199.85331


In [13]:
features = [
    'market-price',
    'difficulty',
    'total-bitcoins',
    'blocks-size',
    'market-cap']
#    0   Unnamed: 0        1499 non-null   int64  
#  1   date              1499 non-null   object 
#  2   market-price      1499 non-null   float64
#  3   difficulty        1499 non-null   float64
#  4   total-bitcoins    1499 non-null   float64
#  5   blocks-size       1499 non-null   float64
#  6   market-cap        1499 non-null   float64
#  7   open              1499 non-null   float64
#  8   high              1499 non-null   float64
#  9   low               1499 non-null   float64
#  10  close             1499 non-null   float64
#  11  volume            1499 non-null   float64
#  12  Number of Trades  1499 non-null   int64  
#  13  TB Base Volume    1499 non-null   float64
#  14  close_nextday

In [14]:
import requests
import warnings
import pandas as pd
import datetime

# Ignore FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning)

url = 'https://api.blockchain.info/charts'
params = {
    'timespan': '1days',
    'format': 'json'
}

# Define the data to retrieve
data_types = [
    'market-price',
    'difficulty',
    'total-bitcoins',
    'blocks-size',
    'market-cap']

# Initialize a dictionary to store the dataframes
dfs = {}

# Loop over the data types and retrieve the data
for data_type in data_types:
    # Set the API endpoint for the current data type
    current_url = f"{url}/{data_type}"
    
    # Make the API request
    response = requests.get(current_url, params=params)
    
    # Extract the data from the response
    data = response.json()['values']
    
    # Get the most recent entry from yesterday
    yesterday = datetime.datetime.now().date() - datetime.timedelta(days=1)
    recent_entry = next((entry for entry in data if datetime.datetime.fromtimestamp(entry['x']).date() == yesterday), None)
    
    # Create a dataframe for the current data type
    df = pd.DataFrame(columns=['date', data_type])
    
    # Add the most recent entry to the dataframe
    if recent_entry:
        df = df.append({'date': datetime.datetime.fromtimestamp(recent_entry['x']).date(), data_type: recent_entry['y']}, ignore_index=True)
    
    # Store the DataFrame in the dictionary
    dfs[data_type] = df

# Print the DataFrames
for data_type, df in dfs.items():
    print(f"{data_type}:")
    print(df)
    print("\n")

market-price:
         date  market-price
0  2023-05-29       28082.7


difficulty:
         date    difficulty
0  2023-05-29  4.954970e+13


total-bitcoins:
         date  total-bitcoins
0  2023-05-29     19386631.25


blocks-size:
         date    blocks-size
0  2023-05-29  484328.151344


market-cap:
         date    market-cap
0  2023-05-29  5.474397e+11




In [15]:
#list of dataframes; each with the blockchain data
frames=[]
for key in dfs.keys():
    name=key.replace('-','')
    name=dfs[key]
    frames.append(name)
frames[4]

,date,market-cap
0,2023-05-29,5.474397e+11


In [16]:
dfs.keys()

dict_keys(['market-price', 'difficulty', 'total-bitcoins', 'blocks-size', 'market-cap'])

In [17]:
#MERGE ALL RESULTING DATAFRAMES 
merged_df=frames[0].merge(frames[1],on='date')
print(merged_df)
for i in range(1,len(frames)-1):
    merged_df=merged_df.merge(frames[i+1],on='date')
    print(merged_df)

         date  market-price    difficulty
0  2023-05-29       28082.7  4.954970e+13
         date  market-price    difficulty  total-bitcoins
0  2023-05-29       28082.7  4.954970e+13     19386631.25
         date  market-price    difficulty  total-bitcoins    blocks-size
0  2023-05-29       28082.7  4.954970e+13     19386631.25  484328.151344
         date  market-price    difficulty  total-bitcoins    blocks-size  \
0  2023-05-29       28082.7  4.954970e+13     19386631.25  484328.151344   

     market-cap  
0  5.474397e+11  


In [18]:
merged_df.set_index('date')

,market-price,difficulty,total-bitcoins,blocks-size,market-cap
date,,,,,
2023-05-29,28082.7,4.954970e+13,19386631.25,484328.151344,5.474397e+11


In [19]:
whole_df=merged_df.merge(hist_df,on='date')

In [20]:
whole_df

,date,market-price,difficulty,total-bitcoins,blocks-size,market-cap,open,high,low,close,volume,Number of Trades,TB Base Volume
0,2023-05-29,28082.7,4.954970e+13,19386631.25,484328.151344,5.474397e+11,28065.01,28447.14,27524.6,27736.4,42385.41945,805579,20199.85331


In [21]:
whole_df.set_index('date',inplace=True)

In [22]:
whole_df

,market-price,difficulty,total-bitcoins,blocks-size,market-cap,open,high,low,close,volume,Number of Trades,TB Base Volume
date,,,,,,,,,,,,
2023-05-29,28082.7,4.954970e+13,19386631.25,484328.151344,5.474397e+11,28065.01,28447.14,27524.6,27736.4,42385.41945,805579,20199.85331


In [50]:
whole_df['close_nextday']=0
whole_df

,market-price,difficulty,total-bitcoins,blocks-size,market-cap,open,high,low,close,volume,Number of Trades,TB Base Volume,close_nextday
date,,,,,,,,,,,,,
2023-05-29,28082.7,4.954970e+13,19386631.25,484328.151344,5.474397e+11,28065.01,28447.14,27524.6,27736.4,42385.41945,805579,20199.85331,0


In [51]:
import pickle
with open('BTCscaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [52]:
btc_scaled = scaler.transform(whole_df)
btc_scaled

array([[ 0.38619988,  1.        ,  1.00375509,  1.00827965,  0.40446037,
         0.38630757,  0.38297778,  0.3863934 ,  0.38478573,  0.04756104,
         0.04903953,  0.04535775, -0.05039104]])

In [54]:
X_test=btc_scaled[:,:12]

In [55]:
import h5py
from tensorflow.keras.models import load_model
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
# Load the saved model
model = load_model('my_bitcoinLSTM_model.h5',custom_objects={'root_mean_squared_error': root_mean_squared_error})

In [56]:
X_test

array([[0.38619988, 1.        , 1.00375509, 1.00827965, 0.40446037,
        0.38630757, 0.38297778, 0.3863934 , 0.38478573, 0.04756104,
        0.04903953, 0.04535775]])

In [57]:
X_test

array([[0.38619988, 1.        , 1.00375509, 1.00827965, 0.40446037,
        0.38630757, 0.38297778, 0.3863934 , 0.38478573, 0.04756104,
        0.04903953, 0.04535775]])

In [58]:
X_test.shape

(1, 12)

In [59]:
import pickle 
with open('bitcoin_closemax.pkl','rb') as file:
    closeMax=pickle.load(file)
    print(closeMax)
with open('bitcoin_closemin.pkl','rb') as file:
    closeMin=pickle.load(file)
    print(closeMin)

66947.66
3211.72


In [60]:
import tensorflow as tf
X_test=tf.expand_dims(X_test,axis=1)

In [61]:
X_test

<tf.Tensor: shape=(1, 1, 12), dtype=float64, numpy=
array([[[0.38619988, 1.        , 1.00375509, 1.00827965, 0.40446037,
         0.38630757, 0.38297778, 0.3863934 , 0.38478573, 0.04756104,
         0.04903953, 0.04535775]]])>

In [62]:
X_test.shape

TensorShape([1, 1, 12])

In [63]:
close_nextday = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [64]:
close_nextday

array([[0.38234255]], dtype=float32)

In [65]:
close_nextday[-1][0]

0.38234255

In [66]:
denormalized_value =  abs((close_nextday[-1][0] * (closeMax - closeMin)+ closeMin))
denormalized_value

27580.681646401288

In [38]:
import subprocess

In [39]:
profitable_coin="bitcoin"

In [40]:
# cmd_bitcoin = "vsim -c -do bscript.do"
# #cmd_litecoin = "vsim -c -do coin2.tcl"

In [41]:
# if profitable_coin == 'bitcoin':
#     subprocess.run(cmd_bitcoin, shell=True)
# elif profitable_coin == 'coin2':
#     subprocess.run(cmd_coin2, shell=True)

###  selects device (index) + programs device (check mode)

In [42]:
def program_device(sof_file,device_index):
    sof_file = "fpgaminer_top.sof"
    device_index = "1"

    # Create the command to program the device
    command = ['quartus_pgm', '-m', 'jtag', '-o', f'p;{sof_file}@{device_index}']

    # Run the command
    subprocess.run(command)


### navigate to .sof directory 

In [43]:
import os

def change_directory(directory):
    os.chdir(directory)

# Call the function to change the directory
#change_directory(r"C:\intelFPGA_lite\16.1\latestBminer\output_files")

In [44]:
def run_quartus_script(script):
    # Run the Quartus script using the Quartus command
    subprocess.run(['quartus_sh', '-t', script+'.tcl'])

# Call the function to run the Quartus script
#run_quartus_script()

In [45]:
if profitable_coin=='bitcoin':
    change_directory(r"C:\intelFPGA_lite\16.1\latestBminer\output_files")
    program_device("fpgamminer_top.sof","1")

In [46]:
# def check_programming_status():
#     # Run the quartus_pgm command with the appropriate options
#     cmd = ['quartus_pgm', '-m', 'jtag', '-l']
#     result = subprocess.run(cmd, capture_output=True, text=True)

#     # Check the output to determine the programming status
#     output = result.stdout.strip()
#     if 'Programmed' in output:
#         print("Device is already programmed.")
#     else:
#         print("Device is not programmed.")

# # Call the function to check the programming status
# check_programming_status()